<a href="https://colab.research.google.com/github/AkimzhanRakhimov/Neural-Network-Potentials-for-QM9-Dataset/blob/main/notebooks/NNP_and_Verlet_Integrator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ase torch-geometric rdkit

In [ ]:
from rdkit import Chem
import torch
import ase
from ase import Atoms
from google.colab import drive
import os
from ase.io import write
import numpy as np
from torch_geometric.datasets import QM9
from torch.utils.data import Dataset,DataLoader
from torch import nn
from tqdm import tqdm


print("Imports ok")

Imports ok


In [ ]:
drive.mount("/content/drive")
if os.path.exists("/content/drive/MyDrive/NNP"):
  print("Path exist already")
else:
  os.mkdir("/content/drive/MyDrive/NNP")
  print("Path has been created successfully")


Mounted at /content/drive
Path exist already


In [ ]:
dataset=QM9(root="qm9_data")

Extracting qm9_data/raw/qm9.zip
Processing...
100%|██████████| 133885/133885 [03:06<00:00, 718.05it/s]
Done!


130831


In [ ]:
class Bounded_Dataset(Dataset):
  def __init__(self,dataset_name,n_samples=10000):
    self.data=dataset_name[:n_samples]

  def __len__(self):
    return len(self.data)

  def __getitem__(self,idx):
    d=self.data[idx]
    return{
        "z":d.z.float(),
        "pos":d.pos.float(),
        "energy":d.y[0,10].float()
    }
training_dataset=Bounded_Dataset(dataset)

In [ ]:
device="cuda" if torch.cuda.is_available() else "cpu"
torch.manual_seed(42)
torch.cuda.manual_seed(42)

class AtomMLP(nn.Module):
  def __init__(self,n_atom_types=100,hidden_dim=64,output_features=1):
    super().__init__()
    self.embeddings=nn.Embedding(n_atom_types,embedding_dim=64)
    # Pytorch architecture allows us to use bathes(atoms in molecula) in neural network loop
    self.mlp=nn.Sequential(
        nn.Linear(hidden_dim+15,hidden_dim),
        nn.ReLU(),
        nn.Linear(hidden_dim,1)
    )
  def forward(self,z,pos):


    dists=torch.cdist(pos,pos)
    N=pos.shape[1]

    mask=1-torch.eye(N,device=pos.device)
    dists=dists*mask
    dists_sorted,_=torch.sort(dists,dim=2)
    if N<=15:
      zeros=torch.zeros((1,N,16-N),device=pos.device)
      dists_sorted=torch.cat([dists_sorted,zeros],dim=2)

    dists_sorted=dists_sorted[:,:,1:16]

    h=self.embeddings(z.long())
    atom_input=torch.cat([h,dists_sorted],dim=2)
    energy=self.mlp(atom_input).sum()
    return energy

# Here i create GNN architecture

class AtomEmbeddings(nn.Module):
  def __init__(self,num_types,hidden_dim):
    super().__init__()
    self.embedding=nn.Embedding(num_types,hidden_dim)

  def forward(self,z):
    h=self.embedding(z.long())
    return h

class MessagePassingLayerV0(nn.Module):
  def __init__(self,hidden_dim):
    super().__init__()
    self.mlp=nn.Sequential(
        nn.Linear(2*hidden_dim+1,hidden_dim),
        nn.ReLU(),
        nn.Linear(hidden_dim,hidden_dim)
    )
  def forward(self,h,edge_index,edge_attr):
    m=torch.zeros_like(h).to(device)
    for idx in range(edge_index.shape[1]):
      i,j=edge_index[:,idx]
      msg_input=torch.cat([h[i],h[j],edge_attr[idx]],dim=0)
      m[i]+=self.mlp(msg_input)
    h=h+m
    return h
class MessagePassingLayerV1(nn.Module):
  def __init__(self,hidden_dim):
    super().__init__()
    self.mlp=nn.Sequential(
        nn.Linear(hidden_dim+1,hidden_dim),
        nn.ReLU(),
        nn.Linear(hidden_dim,hidden_dim)
    )
  def forward(self,h,edge_index,edge_attr):
    m=torch.zeros_like(h)
    for idx in range(edge_index.shape[1]):
      i,j=edge_index[:,idx]
      msg_input=torch.cat([h[j],edge_attr[idx]])
      m[i]+=self.mlp(msg_input)
    h=h+m
    return h
class EnergyHead(nn.Module):
  def __init__(self,hidden_dim):
    super().__init__()
    self.mlp=nn.Sequential(
        nn.Linear(hidden_dim,hidden_dim),
        nn.ReLU(),
        nn.Linear(hidden_dim,1)
    )
  def forward(self,h):
    return self.mlp(h).sum()
class AtomGNN(nn.Module):
  def __init__(self,num_types,hidden_dim,layers_num=3):
    super().__init__()
    self.embedding=AtomEmbeddings(num_types,hidden_dim)
    self.embedding.to(device)
    self.layers=nn.ModuleList([MessagePassingLayerV0(hidden_dim) for _ in range(layers_num)])
    for layer in self.layers:
      layer.to(device)
    self.energy=EnergyHead(hidden_dim)
    self.energy.to(device)
  def forward(self,z,pos,cutoff=5.0):
    N=pos.shape[1]
    dists=torch.cdist(pos,pos).squeeze(0)
    h=self.embedding(z).squeeze(0)
    mask=(dists<cutoff) & (~torch.eye(N, dtype=bool,device=device ))
    edge_index=mask.nonzero(as_tuple=False).T
    edge_attr=dists[mask].unsqueeze(1)
    for layer in self.layers:
      h=layer(h,edge_index,edge_attr)
    return self.energy(h)


In [ ]:
# create a model
model=AtomMLP()
# model=AtomGNN(10,64,3)
model.to(device)

AtomMLP(
  (embeddings): Embedding(100, 64)
  (mlp): Sequential(
    (0): Linear(in_features=79, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=1, bias=True)
  )
)

In [ ]:

train_dataloader=DataLoader(dataset=training_dataset,batch_size=1,shuffle=True)

optimizer=torch.optim.Adam(params=model.parameters(),lr=1e-2)
loss_fn=nn.MSELoss()

In [ ]:
for epoch in range(50):
  model.train()
  total_loss=0
  for batch in tqdm(train_dataloader):
    z=batch['z'].to(device)
    pos=batch['pos'].to(device)
    energy=batch['energy'].to(device)
    # forward pass

    energy_pred=model(z,pos)
    # count the loss
    loss=loss_fn(energy_pred,energy)
    # zero grad
    optimizer.zero_grad()
    # backprop
    loss.backward()
    # update the weights
    optimizer.step()
    total_loss+=loss
  print(f"Loss:{total_loss/len(train_dataloader)}")


In [ ]:
torch.save(obj=model.state_dict(),f="/content/drive/MyDrive/NNP/model_1.pth")
print("Model has been saved")

Model has been saved


In [ ]:
# model.load_state_dict(torch.load("/content/drive/MyDrive/NNP/model_1.pth"))
model.load_state_dict(torch.load("/content/drive/MyDrive/NNP/model_1.pth"))
print("Model has been loaded")
print(model.state_dict())

Model has been loaded
OrderedDict({'embeddings.weight': tensor([[-0.1438,  0.5092, -0.9404,  ...,  1.7845,  0.6640, -1.3630],
        [-2.4863,  5.8489,  2.3534,  ..., -5.7325,  0.1760, -2.6862],
        [-0.0813,  2.6949,  0.5698,  ..., -2.8501, -0.6369,  1.9235],
        ...,
        [-1.2195,  0.0170, -0.5504,  ..., -1.2942,  0.8812,  1.6265],
        [-1.4461,  0.5560,  0.7835,  ...,  0.7141,  0.5716, -0.6407],
        [-1.7449, -1.0938,  0.6311,  ...,  1.7004, -0.9643, -1.0664]]), 'mlp.0.weight': tensor([[-0.6601, -0.5862, -0.3912,  ..., -0.6122, -0.3542, -0.4546],
        [-1.3509, -0.2140, -3.6609,  ..., -2.6910, -3.1587, -3.2637],
        [-5.0041, -5.7155, -0.9885,  ..., -2.2931, -2.3549, -2.2482],
        ...,
        [-2.7026, -4.2215, -2.9582,  ..., -5.2760, -4.9126, -4.3879],
        [-4.3435, -0.3502, -9.8305,  ..., -5.2977, -8.7898, -8.7519],
        [-4.3830, -5.7389, -5.7398,  ..., -6.1473, -6.0429, -5.6927]]), 'mlp.0.bias': tensor([ -0.6764,  -4.7322,  -2.1416,  -0.25

In [ ]:
def get_masses(all_masses):
  masses=[atomic_masses[mass] for mass in all_masses]
  return torch.tensor(masses).unsqueeze(dim=1)
atomic_masses={1:1.0,6:12.0,7:14.0,8:16.0,9:19.0}
batch=next(iter(train_dataloader))
z=batch["z"]
pos=batch["pos"]
pos.requires_grad=True
z_list=list(map(int,z.squeeze(dim=0).tolist()))
masses=get_masses(z_list)
atoms=Atoms(numbers=z_list,positions=pos.squeeze().detach().numpy())
traj=[]
print(atoms.positions)
def integrator(model,masses,z,pos,u,dt=0.002):
  pos.requires_grad_=True
  energy_pred=model(z,pos)

  forces=-torch.autograd.grad(energy_pred,pos,create_graph=True)[0].squeeze(dim=0)
  acc=forces/masses
  u_new=u+0.5*acc*dt

  pos_new=pos+u_new*dt
  return pos_new,u_new
pos_new=pos
u_new=0



[[-0.0105      1.45840001  0.0083    ]
 [ 0.0064      0.         -0.0288    ]
 [-0.1003     -0.69669998 -1.19509995]
 [-0.081      -2.05679989 -1.19350004]
 [ 0.0448     -2.7723999  -0.0355    ]
 [ 0.147      -2.11409998  1.07969999]
 [ 0.1393     -0.65820003  1.20879996]
 [ 0.2343     -0.0259      2.2486999 ]
 [ 0.91500002  1.82840002  0.45570001]
 [-0.1174      1.84200001 -1.00660002]
 [-0.83929998  1.80359995  0.63099998]
 [-0.1991     -0.1106     -2.1013    ]
 [-0.1654     -2.61459994 -2.11739993]
 [ 0.24779999 -2.66659999  2.01090002]]


In [ ]:
for step in range(500):
  traj.append(atoms.copy())
  pos_new,u_new=integrator(model,masses,z,pos_new,u_new,dt=0.02)
  atoms=Atoms(numbers=z_list,positions=pos_new.squeeze().cpu().clone().detach().numpy())

write("/content/drive/MyDrive/NNP/Methan_dynamic.xyz",traj)